# Emergent Particles from TNFR: Photon-like and Electron-like Patterns
This notebook constructs two minimal TNFR patterns on a grid and analyzes them via the canonical structural fields (|∇φ|, K_φ, ξ_C) and a topological winding number Q:

- Photon-like: a plane-wave phase pattern (massless-like propagation mode proxy)
- Electron-like: a localized vortex (soliton/topological defect proxy)

We do not integrate time here; instead, we design static patterns and evaluate their structural signatures.

In [1]:
# Setup: seeds and project path
import sys, os, math, json
from pathlib import Path
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import random
random.seed(42)
np.random.seed(42)

# Ensure repo src/ is importable
cwd = Path.cwd()
repo_root = None
for base in [cwd] + list(cwd.parents):
    if (base / 'src' / 'tnfr').exists() or (base / 'pyproject.toml').exists():
        repo_root = base
        break
SRC = (repo_root / 'src') if repo_root else (cwd / 'src')
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print('Project src set:', SRC if SRC.exists() else 'NOT FOUND')

# Results directory
RESULTS_DIR = Path('results') / 'particles'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
print('Results dir:', RESULTS_DIR)

Project src set: c:\TNFR-Python-Engine\src
Results dir: results\particles


In [2]:
# Imports
import networkx as nx
from tnfr.physics.fields import (
    compute_phase_gradient,
    compute_phase_curvature,
    estimate_coherence_length,
    compute_phase_winding,
)
print('Imports OK')

Imports OK


In [3]:
# Build grid graph with tuple nodes (i, j) for easy geometry
SIDE = 32  # modest size for fast runs
G = nx.grid_2d_graph(SIDE, SIDE)  # nodes are (i, j)

# Initialize telemetry baseline
for n in G.nodes():
    G.nodes[n]['theta'] = 0.0
    G.nodes[n]['phase'] = 0.0
    G.nodes[n]['delta_nfr'] = 0.05
    G.nodes[n]['dnfr'] = 0.05
    G.nodes[n]['coherence'] = 1.0 / (1.0 + 0.05)

print('Grid built:', G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges')

Grid built: 1024 nodes, 1984 edges


## Photon-like pattern: Plane wave on the grid
We impose a plane wave phase φ(i,j) = kx·i + ky·j and evaluate |∇φ| and K_φ statistics.

In [4]:
# Define a plane wave pattern
kx, ky = (0.25, 0.0)  # along x for clarity
for (i, j) in G.nodes():
    phi = (kx * i + ky * j) % (2 * math.pi)
    G.nodes[(i, j)]['theta'] = float(phi)
    G.nodes[(i, j)]['phase'] = float(phi)

# Evaluate fields
grad = compute_phase_gradient(G)
kphi = compute_phase_curvature(G)
grad_vals = np.array(list(grad.values()), dtype=float)
kphi_vals = np.array(list(kphi.values()), dtype=float)

summary_plane = {
    'pattern': 'plane_wave',
    'side': SIDE,
    'kx': kx, 'ky': ky,
    'grad_mean': float(np.mean(grad_vals)) if grad_vals.size else 0.0,
    'grad_std': float(np.std(grad_vals)) if grad_vals.size else 0.0,
    'kphi_mean': float(np.mean(kphi_vals)) if kphi_vals.size else 0.0,
    'kphi_absmax': float(np.max(np.abs(kphi_vals))) if kphi_vals.size else 0.0,
}
with open(RESULTS_DIR / 'plane_wave_summary.json', 'w') as f:
    json.dump(summary_plane, f, indent=2)
print('Plane wave summary:', summary_plane)

Plane wave summary: {'pattern': 'plane_wave', 'side': 32, 'kx': 0.25, 'ky': 0.0, 'grad_mean': 0.125, 'grad_std': 0.0142636082683637, 'kphi_mean': 0.0, 'kphi_absmax': 0.125}


## Electron-like pattern: Localized vortex (topological defect)
We set φ(i,j) = atan2(j−cy, i−cx) producing a 2π winding around the center, then measure K_φ hotspots, ξ_C, and compute the winding number Q on a small square loop.

In [5]:
# Center and vortex phase
cx, cy = SIDE // 2, SIDE // 2
for (i, j) in G.nodes():
    ang = math.atan2((j - cy), (i - cx))  # range (-pi, pi]
    phi = (ang + 2 * math.pi) % (2 * math.pi)
    G.nodes[(i, j)]['theta'] = float(phi)
    G.nodes[(i, j)]['phase'] = float(phi)
    # Set ΔNFR larger near the core and decaying outward
    r = math.hypot(i - cx, j - cy)
    dnfr = 0.2 * math.exp(-r / 4.0) + 0.05
    G.nodes[(i, j)]['delta_nfr'] = float(dnfr)
    G.nodes[(i, j)]['dnfr'] = float(dnfr)
    G.nodes[(i, j)]['coherence'] = 1.0 / (1.0 + abs(dnfr))

# Evaluate fields
kphi_vort = compute_phase_curvature(G)
kphi_vort_vals = np.array(list(kphi_vort.values()), dtype=float)
hotspots = int(np.sum(np.abs(kphi_vort_vals) >= 3.0))
xi_c = float(estimate_coherence_length(G, coherence_key='coherence'))

# Build a small square cycle around the center to compute Q
r0 = 3
cycle = []
# Top edge (left to right)
for x in range(cx - r0, cx + r0 + 1):
    if (x, cy - r0) in G: cycle.append((x, cy - r0))
# Right edge (top to bottom, excluding corner we already added)
for y in range(cy - r0 + 1, cy + r0 + 1):
    if (cx + r0, y) in G: cycle.append((cx + r0, y))
# Bottom edge (right to left)
for x in range(cx + r0 - 1, cx - r0 - 1, -1):
    if (x, cy + r0) in G: cycle.append((x, cy + r0))
# Left edge (bottom to top, excluding corner)
for y in range(cy + r0 - 1, cy - r0, -1):
    if (cx - r0, y) in G: cycle.append((cx - r0, y))

Q = compute_phase_winding(G, cycle) if len(cycle) >= 4 else 0

summary_vortex = {
    'pattern': 'vortex',
    'side': SIDE,
    'center': [cx, cy],
    'kphi_absmax': float(np.max(np.abs(kphi_vort_vals))) if kphi_vort_vals.size else 0.0,
    'hotspots_ge_3': hotspots,
    'xi_C': xi_c,
    'cycle_len': len(cycle),
    'Q': int(Q),
}
with open(RESULTS_DIR / 'vortex_summary.json', 'w') as f:
    json.dump(summary_vortex, f, indent=2)
print('Vortex summary:', summary_vortex)

Vortex summary: {'pattern': 'vortex', 'side': 32, 'center': [16, 16], 'kphi_absmax': 2.356194490192345, 'hotspots_ge_3': 0, 'xi_C': 23934.82431825856, 'cycle_len': 24, 'Q': 1}


In [6]:
# Minimal assertions (soft)
issues = []
# Plane wave sanity
if not (os.path.exists(RESULTS_DIR / 'plane_wave_summary.json')):
    issues.append('Plane wave summary file missing')
# Vortex sanity
if not (os.path.exists(RESULTS_DIR / 'vortex_summary.json')):
    issues.append('Vortex summary file missing')

if issues:
    print('WARN:', issues)
else:
    print('All summaries written.')

All summaries written.
